In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns


from matplotlib import pyplot as plt

In [2]:
program = "maximal_matching"
graph_name = "graph_7"

no_simulations = 10000
fault_interval = 2
sched = 0

selected_nodes = [0, 4, 9]

merge_files = [
    f"w_exp_{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{n}__duong"
    for n in selected_nodes
]

In [3]:
dfs = [pd.read_csv(f"{os.path.join("results", program, f)}.csv").drop('Count', axis=1) for f in merge_files]
for df in dfs:
    df['Steps'] = np.ceil(df['Steps']).astype(int)

In [4]:
dfs[2]

,Steps,exp_fit
0,5,57.611150
1,6,57.288068
2,7,56.966792
3,8,56.647312
4,9,56.329620
...,...,...
2073,2074,-0.159263
2074,2075,-0.159266
2075,2076,-0.159268
2076,2077,-0.159271


In [5]:
dfs[0]

,Steps,exp_fit
0,12,72.342522
1,13,71.827245
2,14,71.315629
3,15,70.807646
4,16,70.303271
...,...,...
1435,1436,-0.200354
1436,1437,-0.200372
1437,1438,-0.200391
1438,1439,-0.200409


In [6]:
dfs[1]

,Steps,exp_fit
0,10,61.743761
1,11,61.369228
2,12,60.996961
3,13,60.626948
4,14,60.259174
...,...,...
1751,1752,-0.141952
1752,1753,-0.141962
1753,1754,-0.141971
1754,1755,-0.141979


In [7]:
df_merged = dfs[0].copy()
for i in range(1, len(dfs)):
    df_merged = pd.merge(
        df_merged,
        dfs[i],
        on=["Steps"],
        suffixes=(f"__{selected_nodes[i - 1]}", ""),
    ).fillna(0)

df_merged.rename(columns={df_merged.columns[-1]: f'exp_fit__{selected_nodes[-1]}'}, inplace=True)
df_merged = df_merged.set_index('Steps')
df_merged

,exp_fit__0,exp_fit__4,exp_fit__9
Steps,,,
12,72.342522,60.996961,55.387163
13,71.827245,60.626948,55.076518
14,71.315629,60.259174,54.767610
15,70.807646,59.893625,54.460430
16,70.303271,59.530289,54.154967
...,...,...,...
1436,-0.200354,-0.133190,-0.141198
1437,-0.200372,-0.133252,-0.141302
1438,-0.200391,-0.133313,-0.141405


In [8]:
for col in df_merged.columns:
    df_merged.rename(columns={col: f"Exponential fit of node {col.split('__')[1]}"}, inplace=True)

df_merged

,Exponential fit of node 0,Exponential fit of node 4,Exponential fit of node 9
Steps,,,
12,72.342522,60.996961,55.387163
13,71.827245,60.626948,55.076518
14,71.315629,60.259174,54.767610
15,70.807646,59.893625,54.460430
16,70.303271,59.530289,54.154967
...,...,...,...
1436,-0.200354,-0.133190,-0.141198
1437,-0.200372,-0.133252,-0.141302
1438,-0.200391,-0.133313,-0.141405


In [9]:
fontsize = 20

def get_title():
    return f"Simulation curve fit - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"


In [10]:
# sns.lineplot(df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)

In [16]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)


ax.set_xlabel("Steps")
ax.set_ylabel("Count")

plt.rc("font", size=fontsize)
plt.legend(fontsize=fontsize * 0.9)

plt.savefig(
    f"exp_curve_fit_{program}__{graph_name}__{''.join([str(i) for i in selected_nodes])}.png",
    bbox_inches="tight",
)
plt.close()